In [ ]:
# https://rguigoures.github.io/word2vec_pytorch/

In [2]:
import re
import nltk
nltk.download('brown')
from nltk.corpus import brown
import itertools
corpus = []

for cat in ['news']:
    for text_id in brown.fileids(cat):
        raw_text = list(itertools.chain.from_iterable(brown.sents(text_id)))
        text = ' '.join(raw_text)
        text = text.lower()
        text.replace('\n', ' ')
        text = re.sub('[^a-z ]+', '', text)
        corpus.append([w for w in text.split() if w != ''])

[nltk_data] Downloading package brown to /home/openpose/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
#Subsample frequent words

from collections import Counter
import random, math

def subsample_frequent_words(corpus):
    filtered_corpus = []
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    sum_word_counts = sum(list(word_counts.values()))
    word_counts = {word: word_counts[word]/float(sum_word_counts) for word in word_counts}
    for text in corpus:
        filtered_corpus.append([])
        for word in text:
            if random.random() < (1+math.sqrt(word_counts[word] * 1e3)) * 1e-3 / float(word_counts[word]):
                filtered_corpus[-1].append(word)
    return filtered_corpus


In [4]:
corpus = subsample_frequent_words(corpus)
vocabulary = set(itertools.chain.from_iterable(corpus))

word_to_index = {w: idx for (idx, w) in enumerate(vocabulary)}
index_to_word = {idx: w for (idx, w) in enumerate(vocabulary)}


In [5]:
#Bag of words
import numpy as np

context_tuple_list = []
w = 4

for text in corpus:
    for i, word in enumerate(text):
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j]))
print("There are {} pairs of target and context words".format(len(context_tuple_list)))



There are 475002 pairs of target and context words


In [ ]:
# import torch
# import torch.nn  as  nn
# import torch.autograd as autograd
# import torch.optim as optim
# import torch.nn.functional as F


# class Word2Vec(nn.Module):

#     def __init__(self, embedding_size, vocab_size):
#         super(Word2Vec, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_size)
#         self.linear = nn.Linear(embedding_size, vocab_size)
        
#     def forward(self, context_word):
#         emb = self.embeddings(context_word)
#         hidden = self.linear(emb)
#         out = F.log_softmax(hidden)
#         return out


In [6]:
# Stop loss if it is not improoving
class EarlyStopping():
    def __init__(self, patience=5, min_percent_gain=0.1):
        self.patience = patience
        self.loss_list = []
        self.min_percent_gain = min_percent_gain / 100.
        
    def update_loss(self, loss):
        self.loss_list.append(loss)
        if len(self.loss_list) > self.patience:
            del self.loss_list[0]
    
    def stop_training(self):
        if len(self.loss_list) == 1:
            return False
        gain = (max(self.loss_list) - min(self.loss_list)) / max(self.loss_list)
        print("Loss gain: {}%".format(round(100*gain,2)))
        if gain < self.min_percent_gain:
            return True
        else:
            return False

In [7]:
#Learning
# vocabulary_size = len(vocabulary)

# net = Word2Vec(embedding_size=2, vocab_size=vocabulary_size)
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters())
# early_stopping = EarlyStopping()
# context_tensor_list = []

# for target, context in context_tuple_list:
#     target_tensor = autograd.Variable(torch.LongTensor([word_to_index[target]]))
#     context_tensor = autograd.Variable(torch.LongTensor([word_to_index[context]]))
#     context_tensor_list.append((target_tensor, context_tensor))
    
# while True:
#     losses = []
#     for target_tensor, context_tensor in context_tensor_list:
#         net.zero_grad()
#         log_probs = net(context_tensor)
#         loss = loss_function(log_probs, target_tensor)
#         loss.backward()
#         optimizer.step()
#         losses.append(loss.data)
#     print("Loss: ", np.mean(losses))
#     early_stopping.update_loss(np.mean(losses))
#     if early_stopping.stop_training():
#         break


In [8]:

import random

def get_batches(context_tuple_list, batch_size=100):
    random.shuffle(context_tuple_list)
    batches = []
    batch_target, batch_context, batch_negative = [], [], []
    for i in range(len(context_tuple_list)):
        batch_target.append(word_to_index[context_tuple_list[i][0]])
        batch_context.append(word_to_index[context_tuple_list[i][1]])
        batch_negative.append([word_to_index[w] for w in context_tuple_list[i][2]])
        if (i+1) % batch_size == 0 or i == len(context_tuple_list)-1:
            tensor_target = autograd.Variable(torch.from_numpy(np.array(batch_target)).long())
            tensor_context = autograd.Variable(torch.from_numpy(np.array(batch_context)).long())
            tensor_negative = autograd.Variable(torch.from_numpy(np.array(batch_negative)).long())
            batches.append((tensor_target, tensor_context, tensor_negative))
            batch_target, batch_context, batch_negative = [], [], []
    return batches

In [9]:
#artificial negative examples
from numpy.random import multinomial

def sample_negative(sample_size):
    sample_probability = {}
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    normalizing_factor = sum([v**0.75 for v in word_counts.values()])
    for word in word_counts:
        sample_probability[word] = word_counts[word]**0.75 / normalizing_factor
    words = np.array(list(word_counts.keys()))
    while True:
        word_list = []
        sampled_index = np.array(multinomial(sample_size, list(sample_probability.values())))
        for index, count in enumerate(sampled_index):
            for _ in range(count):
                 word_list.append(words[index])
        yield word_list


In [10]:
# make this threaded
import numpy as np

context_tuple_list = []
w = 4
negative_samples = sample_negative(8)

for text in corpus:
    for i, word in enumerate(text):
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j], next(negative_samples)))
print("There are {} pairs of target and context words".format(len(context_tuple_list)))


There are 475002 pairs of target and context words


In [11]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings_target = nn.Embedding(vocab_size, embedding_size)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_size)

    def forward(self, target_word, context_word, negative_example):
        emb_target = self.embeddings_target(target_word)
        emb_context = self.embeddings_context(context_word)
        emb_product = torch.mul(emb_target, emb_context)
        emb_product = torch.sum(emb_product, dim=1)
        out = torch.sum(F.logsigmoid(emb_product))
        emb_negative = self.embeddings_context(negative_example)
        emb_product = torch.bmm(emb_negative, emb_target.unsqueeze(2))
        emb_product = torch.sum(emb_product, dim=1)
        out += torch.sum(F.logsigmoid(-emb_product))
        return -out


In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import time

vocabulary_size = len(vocabulary)

loss_function = nn.CrossEntropyLoss()
net = Word2Vec(embedding_size=200, vocab_size=vocabulary_size)

optimizer = optim.Adam(net.parameters())
early_stopping = EarlyStopping(patience=5, min_percent_gain=1)

while True:
    losses = []
    context_tuple_batches = get_batches(context_tuple_list, batch_size=2000)
    for i in range(len(context_tuple_batches)):
        net.zero_grad()
        target_tensor, context_tensor, negative_tensor = context_tuple_batches[i]
        loss = net(target_tensor, context_tensor, negative_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)
        
    print("Loss: ", np.mean(losses))
    early_stopping.update_loss(np.mean(losses))
    if early_stopping.stop_training():
        break


Loss:  42203.43
Loss:  37249.58
Loss gain: 11.74%
Loss:  32722.445
Loss gain: 22.46%
Loss:  28558.176
Loss gain: 32.33%
Loss:  24816.186
Loss gain: 41.2%
Loss:  21544.676
Loss gain: 42.16%
Loss:  18747.59
Loss gain: 42.71%
Loss:  16378.151
Loss gain: 42.65%
Loss:  14366.645
Loss gain: 42.11%
Loss:  12654.258
Loss gain: 41.27%
Loss:  11187.031
Loss gain: 40.33%
Loss:  9922.428
Loss gain: 39.42%
Loss:  8828.775
Loss gain: 38.55%
Loss:  7876.5806
Loss gain: 37.76%
Loss:  7043.5796
Loss gain: 37.04%
Loss:  6313.2974
Loss gain: 36.37%
Loss:  5669.857
Loss gain: 35.78%
Loss:  5099.7017
Loss gain: 35.25%
Loss:  4593.1377
Loss gain: 34.79%
Loss:  4141.332
Loss gain: 34.4%
Loss:  3736.6316
Loss gain: 34.1%
Loss:  3373.6433
Loss gain: 33.85%
Loss:  3046.3271
Loss gain: 33.68%
Loss:  2751.6074
Loss gain: 33.56%
Loss:  2484.887
Loss gain: 33.5%
Loss:  2243.4785
Loss gain: 33.5%
Loss:  2023.9703
Loss gain: 33.56%
Loss:  1824.7343
Loss gain: 33.68%
Loss:  1643.8716
Loss gain: 33.85%
Loss:  1479.9067

In [ ]:
import numpy as np

def get_closest_word(word, topn=5):
    word_distance = []
    emb = net.embeddings_target
    pdist = nn.PairwiseDistance()
    i = word_to_index[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long)
    v_i = emb(lookup_tensor_i)
    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long)
            v_j = emb(lookup_tensor_j)
            word_distance.append((index_to_word[j], float(pdist(v_i, v_j))))
    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]